# Lightweight Engine: NMPT Portfolio Management

This notebook analyzes 11 standard technical indicators using the **Original (Lightweight) Engine** with a **NMPT** allocation approach.

### Indicators Included:
1. OBV (On-Balance Volume)
2. A/D Line (Accumulation/Distribution)
3. RSI (Relative Strength Index)
4. MACD (Moving Average Convergence Divergence)
5. Stochastic Oscillator
6. ADX (Average Directional Index)
7. Aroon Oscillator
8. CCI (Commodity Channel Index)
9. Bollinger Bands
10. Ichimoku Cloud
11. SMA Golden Cross

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from bokeh.io import output_notebook, show

# Setup paths
sys.path.append('../src')
from backtester.engine import run_backtest, BacktestConfig
from backtester.portfolio import equal_weight, optimize_mpt, optimize_nmpt_hrp
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from features.core import (
    adx, aroon, bollinger_bands, cci, ichimoku, macd, 
    obv, accumulation_distribution_line, rsi, sma, stochastic_oscillator
)

output_notebook()

# Load data
data_dir = "../dataset/cleaned"
asset_files = sorted([f for f in os.listdir(data_dir) if f.startswith("Asset_")])[:30]
assets = [f.split('.')[0] for f in asset_files]

dfs = {}
for a, f in zip(assets, asset_files):
    dfs[a] = pd.read_csv(os.path.join(data_dir, f), parse_dates=True, index_col='Date').sort_index()

returns_matrix = pd.DataFrame({a: df['Close'].pct_change() for a, df in dfs.items()})
close_prices = pd.DataFrame({a: df['Close'] for a, df in dfs.items()})

# Build Market Proxy (Equally weighted average of all loaded assets)
market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in dfs.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in dfs.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in dfs.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in dfs.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in dfs.values()], axis=1).sum(axis=1)
}).sort_index()

print(f"Loaded {len(dfs)} assets.")

In [ ]:
def hysteresis_position(enter: pd.Series, exit: pd.Series) -> pd.Series:
    """Latches a signal state: True for Long, False for Out/Exit."""
    pos = pd.Series(False, index=enter.index)
    current = False
    for dt in enter.index:
        if exit.loc[dt]:
            current = False
        elif enter.loc[dt]:
            current = True
        pos.loc[dt] = current
    return pos

def smooth_slope(series: pd.Series, window: int = 20) -> pd.Series:
    return series.diff(1).ewm(span=window).mean()

def get_divergence_signals(price: pd.Series, indicator: pd.Series, window: int = 20) -> pd.Series:
    p_slope = smooth_slope(price, window)
    i_slope = smooth_slope(indicator, window)
    bull_div = (p_slope < 0) & (i_slope > 0)
    bear_div = (p_slope > 0) & (i_slope < 0)
    return hysteresis_position(bull_div, bear_div)

def generate_weights(indicator_logic, pm_style, rebalance_freq='W'):
    """
    Generates weights over time. 
    To optimize speed for MPT/NMPT, we rebalance on a frequency (e.g., Weekly 'W')
    rather than every single day.
    """
    all_dates = returns_matrix.index
    weight_history = []
    
    active_sets = {}
    for a in assets: 
        active_sets[a] = indicator_logic(dfs[a])
    
    # Rebalancing dates
    rebal_dates = pd.Series(all_dates, index=all_dates).resample(rebalance_freq).last().values
    
    last_w_dict = {}
    last_candidates = []
    
    for t in all_dates:
        candidates = [a for a in assets if t in active_sets[a].index and active_sets[a].loc[t]]
        
        # Trigger rebalance if date matches OR candidates set changed
        is_rebal_day = (t in rebal_dates) or (set(candidates) != set(last_candidates))
        
        if is_rebal_day:
            if pm_style == '1N':
                w_dict = equal_weight(candidates)
            elif pm_style == 'MPT':
                w_dict = optimize_mpt(returns_matrix, candidates, t)
            elif pm_style == 'NMPT':
                w_dict = optimize_nmpt_hrp(returns_matrix, candidates, t)
            else:
                w_dict = {}
            last_w_dict = w_dict
            last_candidates = candidates
        else:
            w_dict = last_w_dict
            
        w_row = pd.Series(0.0, index=assets)
        for a, weight in w_dict.items():
            w_row[a] = weight
        weight_history.append(w_row)
        
    return pd.DataFrame(weight_history, index=all_dates)

In [ ]:
def run_and_report(weights, title):
    res = run_backtest(close_prices, weights)
    report = compute_backtest_report(result=res, close_prices=close_prices)
    display(report.to_frame(title))
    
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title
    )
    show(layout)
    return res

## 1. OBV (On-Balance Volume)
Long when bullish divergence identified.

In [ ]:
def obv_logic(df):
    ind = obv(df['Close'], df['Volume'])
    return get_divergence_signals(df['Close'], ind)

weights_obv = generate_weights(obv_logic, 'NMPT')
res_obv = run_and_report(weights_obv, 'OBV Divergence Strategy')

## 2. A/D Line (Accumulation/Distribution)
Long when bullish divergence identified.

In [ ]:
def ad_logic(df):
    ind = accumulation_distribution_line(df['High'], df['Low'], df['Close'], df['Volume'])
    return get_divergence_signals(df['Close'], ind)

weights_ad = generate_weights(ad_logic, 'NMPT')
res_ad = run_and_report(weights_ad, 'A/D Divergence Strategy')

## 3. RSI (Relative Strength Index)
Long when RSI < 30, Exit when RSI > 70.

In [ ]:
def rsi_logic(df):
    r = rsi(df['Close'])
    return hysteresis_position(r < 30, r > 70)

weights_rsi = generate_weights(rsi_logic, 'NMPT')
res_rsi = run_and_report(weights_rsi, 'RSI Strategy')

## 4. MACD
Long when MACD Line > Signal Line.

In [ ]:
def macd_logic(df):
    m = macd(df['Close'])
    return hysteresis_position(m['macd'] > m['macd_signal'], m['macd'] < m['macd_signal'])

weights_macd = generate_weights(macd_logic, 'NMPT')
res_macd = run_and_report(weights_macd, 'MACD Strategy')

## 5. Stochastic Oscillator
Long when %K < 20, Exit when %K > 80.

In [ ]:
def stoch_logic(df):
    s = stochastic_oscillator(df['High'], df['Low'], df['Close'])
    return hysteresis_position(s['stoch_k'] < 20, s['stoch_k'] > 80)

weights_stoch = generate_weights(stoch_logic, 'NMPT')
res_stoch = run_and_report(weights_stoch, 'Stochastic Strategy')

## 6. ADX (Average Directional Index)
Long when ADX > 25 and +DI > -DI.

In [ ]:
def adx_logic(df):
    a = adx(df['High'], df['Low'], df['Close'])
    enter = (a['adx'] > 25) & (a['plus_di'] > a['minus_di'])
    exit = (a['minus_di'] > a['plus_di']) | (a['adx'] < 20)
    return hysteresis_position(enter, exit)

weights_adx = generate_weights(adx_logic, 'NMPT')
res_adx = run_and_report(weights_adx, 'ADX Strategy')

## 7. Aroon Oscillator
Long when Aroon Up > 70.

In [ ]:
def aroon_logic(df):
    ar = aroon(df['High'], df['Low'])
    return hysteresis_position(ar['aroon_up'] > 70, ar['aroon_up'] < 30)

weights_aroon = generate_weights(aroon_logic, 'NMPT')
res_aroon = run_and_report(weights_aroon, 'Aroon Strategy')

## 8. CCI (Commodity Channel Index)
Long when CCI > 100, Exit when CCI < -100.

In [ ]:
def cci_logic(df):
    c = cci(df['High'], df['Low'], df['Close'])
    return hysteresis_position(c > 100, c < -100)

weights_cci = generate_weights(cci_logic, 'NMPT')
res_cci = run_and_report(weights_cci, 'CCI Strategy')

## 9. Bollinger Bands
Long when Price crosses Upper Band (Trend Following).

In [ ]:
def bb_logic(df):
    bb = bollinger_bands(df['Close'])
    return hysteresis_position(df['Close'] > bb['bb_upper'], df['Close'] < bb['bb_mid'])

weights_bb = generate_weights(bb_logic, 'NMPT')
res_bb = run_and_report(weights_bb, 'Bollinger Bands Strategy')

## 10. Ichimoku Cloud
Long when Price > Cloud Top.

In [ ]:
def ichimoku_logic(df):
    ich = ichimoku(df['High'], df['Low'], df['Close'])
    cloud_top = pd.concat([ich['ichimoku_span_a'], ich['ichimoku_span_b']], axis=1).max(axis=1)
    cloud_bottom = pd.concat([ich['ichimoku_span_a'], ich['ichimoku_span_b']], axis=1).min(axis=1)
    return hysteresis_position(df['Close'] > cloud_top, df['Close'] < cloud_bottom)

weights_ich = generate_weights(ichimoku_logic, 'NMPT')
res_ich = run_and_report(weights_ich, 'Ichimoku Strategy')

## 11. SMA Golden Cross
Long when SMA 50 > SMA 200.

In [ ]:
def sma_logic(df):
    s50 = sma(df['Close'], 50)
    s200 = sma(df['Close'], 200)
    return (s50 > s200)

weights_sma = generate_weights(sma_logic, 'NMPT')
res_sma = run_and_report(weights_sma, 'SMA Golden Cross Strategy')

## Summary Comparison

In [ ]:
compare = pd.DataFrame({
    'OBV': res_obv.equity, 
    'AD': res_ad.equity, 
    'RSI': res_rsi.equity, 
    'MACD': res_macd.equity,
    'Stoch': res_stoch.equity,
    'ADX': res_adx.equity,
    'Aroon': res_aroon.equity,
    'CCI': res_cci.equity,
    'BB': res_bb.equity,
    'Ichimoku': res_ich.equity,
    'SMA': res_sma.equity
})
compare.plot(figsize=(12, 6), title='Comparison of 11 Indicators with NMPT PM')
plt.show()